In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import re

## First scrape data from one page

In [7]:
# Waset.org address
url = "https://waset.org/aeronautics-and-astronautics-conference-in-february-2022-in-paris"

<Response [410]>

In [11]:
r = requests.get(url, headers={'User-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'})
r

<Response [200]>

In [12]:
# View text
r.text

'<!DOCTYPE html>\n<html lang="en" dir="ltr" id="desktop">\n<head>\n        <!-- Global site tag (gtag.js) - Google Analytics -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-25347189-2"></script>\n    <script>\n    window.dataLayer = window.dataLayer || [];\n    function gtag(){dataLayer.push(arguments);}\n    gtag(\'js\', new Date());\n    gtag(\'config\', \'UA-25347189-2\');\n    </script>\n    <!-- Global site tag (gtag.js) - Google Analytics -->\n        <title>International Conference on Aeronautics and Astronautics ICAA in February 2022 in Paris</title>\n    <meta name="description" content="Aeronautics and Astronautics scheduled on February 17-18, 2022 in February 2022 in Paris is for the researchers, scientists, scholars, engineers, academic, scientific and university practitioners to present research activities that might want to attend events, meetings, seminars, congresses, workshops, summit, and symposiums.">\n    <meta name="keywords" content="aer

In [13]:
# Parse HTML tags
BeautifulSoup(r.text)

<!DOCTYPE html>
<html dir="ltr" id="desktop" lang="en">
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-25347189-2"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());
    gtag('config', 'UA-25347189-2');
    </script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<title>International Conference on Aeronautics and Astronautics ICAA in February 2022 in Paris</title>
<meta content="Aeronautics and Astronautics scheduled on February 17-18, 2022 in February 2022 in Paris is for the researchers, scientists, scholars, engineers, academic, scientific and university practitioners to present research activities that might want to attend events, meetings, seminars, congresses, workshops, summit, and symposiums." name="description"/>
<meta content="aerospace, spacecraft, february 2022 in paris, 2022, 2023, 2024" name="k

In [14]:
conf = BeautifulSoup(r.text)

In [17]:
# Find titles
conf.find_all('title')[0]

<title>International Conference on Aeronautics and Astronautics ICAA in February 2022 in Paris</title>

In [18]:
# Extract string
conf.find_all('title')[0].string

'International Conference on Aeronautics and Astronautics ICAA in February 2022 in Paris'

In [19]:
# Titles of the conferences
conftitle = conf.find_all('title')[0].string

In [25]:
conf.find_all('h2')[0].string

'February 17-18, 2022 in Paris, France'

In [28]:
# Location of the conf.
dateloc = conf.find_all('h2')[0].string.split(' in ')
confdate = dateloc[0]
confloc = dateloc[1]

In [29]:
# Build dictionary of titles, dates, and locations
my_dict = {'title': [conftitle],
           'date':[confdate],
           'location':[confloc],
          }

# Convert to DataFrame table
pd.DataFrame(my_dict)

,title,date,location
0,International Conference on Aeronautics and As...,"February 17-18, 2022","Paris, France"


In [30]:
# Inspect tag "td"
conf.find_all("td")

[<td>Abstracts/Full-Text Paper Submission Deadline</td>,
 <td> </td>,
 <td class="textright">January 13, 2022</td>,
 <td>Notification of Acceptance/Rejection</td>,
 <td> </td>,
 <td class="textright">January 31, 2022</td>,
 <td>Final Paper (Camera Ready) Submission &amp; Early Bird Registration Deadline</td>,
 <td> </td>,
 <td class="textright">January 21, 2022</td>,
 <td>Conference Dates</td>,
 <td> </td>,
 <td class="textright">February 17-18, 2022</td>,
 <td>Non-Student Oral/Poster Presenter Registration</td>,
 <td class="earlyBird0">€ 450</td>,
 <td class="earlyBird1">€ 500</td>,
 <td>Student Oral/Poster Presenter Registration</td>,
 <td class="earlyBird0">€ 350</td>,
 <td class="earlyBird1">€ 400</td>,
 <td>Listener Registration</td>,
 <td class="earlyBird0">€ 250</td>,
 <td class="earlyBird1">€ 300</td>,
 <td>Additional Paper Publication</td>,
 <td colspan="2">€ 100</td>]

In [31]:
# Inspect class 'earlyBird'
conf.find_all('td', 'earlyBird1')

[<td class="earlyBird1">€ 500</td>,
 <td class="earlyBird1">€ 400</td>,
 <td class="earlyBird1">€ 300</td>]

In [32]:
conf.find_all('td', 'earlyBird1')[0]

<td class="earlyBird1">€ 500</td>

In [33]:
conf.find_all('td', 'earlyBird1')[0].string

'€ 500'

In [34]:
confreg = conf.find_all('td', 'earlyBird1')[0].string

In [35]:
# Add registration fee
my_dict = {'title': [conftitle],
           'date':[confdate],
           'location':[confloc],
           'regfee':[confreg]
          }

pd.DataFrame(my_dict)

,title,date,location,regfee
0,International Conference on Aeronautics and As...,"February 17-18, 2022","Paris, France",€ 500


In [39]:
# Inspect submission deadline
subdeadline = conf.find_all('td', 'textright')[0].string

In [40]:
# Add submission deadline
my_dict = {'title': [conftitle],
           'date':[confdate],
           'location':[confloc],
           'regfee':[confreg],
           'sub deadline':[subdeadline]
          }

pd.DataFrame(my_dict)

,title,date,location,regfee,sub deadline
0,International Conference on Aeronautics and As...,"February 17-18, 2022","Paris, France",€ 500,"January 13, 2022"


## Building a Spider

In [9]:
url2 = "https://waset.org/conferences-in-february-2022-in-paris"

In [43]:
r = requests.get(url2)
r

<Response [200]>

In [44]:
conflinks = BeautifulSoup(r.text)

In [45]:
# Inspect conferences
conflinks.find_all("a", title=True)

[<a class="nav-link" href="https://waset.org/conferences" title="Conferences in 2022/2023/2024">Conferences</a>,
 <a class="nav-link" href="https://waset.org/disciplines" title="Disciplines">Disciplines</a>,
 <a class="nav-link" href="https://waset.org/page/support" title="Support">Support</a>,
 <a href="https://waset.org/conferences" title="Conferences in 2022/2023/2024">Conferences</a>,
 <a href="https://waset.org/conferences-in-2022" title="Conferences in 2022">2022</a>,
 <a href="https://waset.org/aeronautics-and-astronautics-conference-in-february-2022-in-paris" title="International Conference on Aeronautics and Astronautics in February 2022 in Paris">
                             ICAA 2022: Aeronautics and Astronautics Conference, <span style="color:#63cb1e">Paris</span>
 </a>,
 <a href="https://waset.org/agriculture-animal-aquaculture-sciences-and-fisheries-conference-in-february-2022-in-paris" title="International Conference on Agriculture, Animal, Aquaculture Sciences and Fish

In [49]:
# Pick up titles from 'href'
links = [j["href"] for j in conflinks.find_all("a", title=True)]

In [50]:
links[5:]

['https://waset.org/aeronautics-and-astronautics-conference-in-february-2022-in-paris',
 'https://waset.org/agriculture-animal-aquaculture-sciences-and-fisheries-conference-in-february-2022-in-paris',
 'https://waset.org/advanced-architectural-and-bridge-engineering-conference-in-february-2022-in-paris',
 'https://waset.org/amino-acid-chemistry-conference-in-february-2022-in-paris',
 'https://waset.org/aeronautical-and-astronautical-engineering-conference-in-february-2022-in-paris',
 'https://waset.org/advances-in-applied-electrical-engineering-conference-in-february-2022-in-paris',
 'https://waset.org/anthropology-archaeology-history-and-philosophy-conference-in-february-2022-in-paris',
 'https://waset.org/advances-in-aerospace-materials-design-conference-in-february-2022-in-paris',
 'https://waset.org/aerospace-avionics-and-systems-conference-in-february-2022-in-paris',
 'https://waset.org/applied-bionics-conference-in-february-2022-in-paris',
 'https://waset.org/biopharmaceutics-and

In [16]:
# Bringing everything from above inside spider() function
def spider(url):
    r = requests.get(url)
    conf = BeautifulSoup(r.text)
    conftitle = conf.find_all('title')[0].string
    dateloc = conf.find_all('h2')[0].string.split(' n ')
    confdate = dateloc[0]
    confloc = dateloc[1]
    confreg = con.find_all('td', 'earlyBird1')[0].string
    my_dict = {'title': [conftitle],
           'date':[confdate],
           'location':[confloc],
           'regfee':[confreg],
           'sub deadline':[subdeadline]
          }

    return pd.DataFrame(my_dict)

In [ ]:
# Loop for the spider to check all the links
df = pd.DataFrame()
for l in links:
    df = df.append(spider(l))